In [1]:
import os
from typing import List, Dict, Any
from raptor_config import RaptorConfig

import sys
from pathlib import Path

# 将项目根目录添加到 sys.path
project_root = str(Path().absolute().parent)  # 根据实际情况调整
sys.path.append(project_root)

# 现在可以正常导入
from indexing.raptor_manager import RaptorManager

d:\Miniconda\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def test_single_method(chunks: List[str], method: str) -> None:
    """测试单个聚类方法"""
    print(f"\n{'='*20} 测试 {method} 聚类方法 {'='*20}")
    
    # 创建配置
    config = RaptorConfig(
        enabled=True,
        clustering_method=method,
        n_levels = 2,
        min_cluster_size=3
    )
    
    # 创建RAPTOR管理器
    manager = RaptorManager(config)
    
    try:
        # 构建层次结构
        hierarchy = manager.build_hierarchy(chunks)
        print("hierarchy:\n", hierarchy)
        # print_hierarchy(hierarchy)
    except Exception as e:
        print(f"错误: {str(e)}")

# def print_hierarchy(hierarchy: Dict[int, Dict[str, Any]]) -> None:
#     """打印层次结构结果"""
#     for level, data in hierarchy.items():
#         print(f"\n层级 {level}:")
#         print(f"文档数量: {len(data['docs'])}")
#         print(f"摘要数量: {len(data['summaries'])}")
#         print("\n示例摘要:")
#         # for i, summary in enumerate(data['summaries'][:2]):  # 只打印前两个摘要
#         #     print(f"摘要 {i+1}: {summary[:100]}...")
#         for i, summary in enumerate(data['summaries']):
#             print(f"摘要 {i+1}: {summary}")
#         print(f"\n父子关系示例:")
#         for summary_id, doc_ids in list(data['parent_child_map'].items())[:2]:  # 只打印前两组关系
#             print(f"摘要ID {summary_id[:8]}... 包含文档: {[did[:8] + '...' for did in doc_ids]}")
def print_hierarchy(hierarchy: Dict[int, Dict[str, Any]]) -> None:
    """打印层次结构结果"""
    for level, data in hierarchy.items():
        print(f"\n{'='*50}")
        print(f"层级 {level}:")
        print(f"文档数量: {len(data['docs'])}")
        print(f"摘要数量: {len(data['summaries'])}")
        print(f"{'='*50}")

        # 遍历每个摘要及其对应的文档
        for i, (summary_id, doc_ids) in enumerate(data['parent_child_map'].items()):
            print(f"\n摘要 {i+1}:")
            print(f"摘要ID: {summary_id[:8]}...")
            print(f"摘要内容: {data['summaries'][i]}")
            print("\n对应的原始文档:")
            
            # 获取原始文档内容
            for j, doc_id in enumerate(doc_ids):
                # 找到文档ID对应的索引
                doc_index = data['doc_ids'].index(doc_id)
                doc_content = data['docs'][doc_index]
                print(f"文档 {j+1}: {doc_content}")
            
            print(f"\n{'-'*50}")  # 分隔不同的摘要组

def main():
    """集成测试主函数"""
    # 设置API密钥和基础URL（如果需要）
    os.environ['ZETATECHS_API_KEY'] = ''
    os.environ['ZETATECHS_API_BASE'] = ''
    
    # 准备测试数据
    chunks = [
    "LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。",
    "LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。",
    "LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。",
    "使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。",
    "LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。",
    "在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。",
    "LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。",
    "对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。",
    "LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。",
    "LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。",
    "LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。",
    "LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。",
    "我是崔鹏",
    "崔鹏",
    "sxy"
]
    
    # 测试不同的聚类方法
    clustering_methods = ['gmm', 'hierarchical', 'dbscan', 'raptor']
    
    for method in clustering_methods:
        test_single_method(chunks, method)

def test_specific_method():
    """测试特定的聚类方法（用于调试）"""
    chunks = [
    "LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。",
    "LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。",
    "LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。",
    "使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。",
    "LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。",
    "在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。",
    "LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。",
    "对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。",
    "LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。",
    "LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。",
    "LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。",
    "LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。",
    "我是崔鹏",
    "崔鹏",
    "sxy",
    "崔鹏是sxy的爸爸",
    "sxy是崔鹏的女儿",
    "崔鹏是sxy的爸爸，sxy是崔鹏的女儿"
]
    
    # 只测试RAPTOR方法
    test_single_method(chunks, 'raptor')

In [3]:
if __name__ == "__main__":
    # main()  # 测试所有方法
    test_specific_method()  # 测试特定方法


==================== 测试 raptor 聚类方法 ====================
Chat请求失败 (尝试 1/3): Error code: 400 - {'error': {'message': 'response_format value as json_schema is enabled only for api versions 2024-08-01-preview and later (request id: 20250214130145113196612boicRFry) (request id: 2025021415314596529251fJJQkckf) (request id: 2025021415314590465129h3Mcm82D) (request id: 20250214153144961474059fhQwMz5D)', 'type': '', 'param': '', 'code': 'BadRequest'}}
hierarchy:
 {0: {'docs': ['LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切

In [20]:
a =  {0: {'docs': ['LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', '我是崔鹏', '崔鹏', 'sxy', '崔鹏是sxy的爸爸', 'sxy是崔鹏的女儿', '崔鹏是sxy的爸爸，sxy是崔鹏的女儿'], 'doc_ids': ['b244097f-e171-4811-8279-67b16082db40', '5bf426e0-2176-4424-8d35-c835cb553523', 'fa51cc37-2665-4d62-a3a8-4ad52af695a0', 'a59fa74e-fe67-4e3c-97c9-d28d96e039c9', 'c900e231-267f-49bd-b544-a26efc5cdaed', 'fd0fbf9c-815d-45b4-a925-021fdb4e5a5f', 'bd45d0f2-7369-443c-977b-6e62f29aa6e8', '86b24532-b010-4d72-8e90-2c5fd9de126e', 'b0ecc3b3-753b-4bca-aff4-43fd47296cbc', 'ad2ca09b-a19f-4aaa-bb8f-4f70e5c6d538', 'b5786d7b-608f-4384-9820-7c663db9b08b', '0574a965-0f2d-42a3-8fa4-9afb39cba54b', '557bd4f1-89bf-493c-8e3a-d4160181b370', 'a2a48024-b664-4e4a-8136-b5a705e43884', '482ac2d9-c2e0-4a37-845a-0cf88d9b5e26', 'e84bd680-fef1-42c4-8215-d4db8a077e8e', '34d80142-06f6-4099-b363-ed13367ce49e', '8af5c05d-f769-4d88-b272-fc43aeefceca'], 'summaries': ['LangChain是一个开发基于语言模型的应用框架，具有提示管理、记忆管理和模型链接等核心功能，助力聊天机器人、文档分析工具和知识库等应用的实现，其灵活性适合多领域场景。', 'LCEL（LangChain Expression Language）是LangChain的重要特性，提供功能类似于管道操作的声明式方法，方便开发者组合组件和创建复杂的工作流。LCEL支持多种执行模式，如串行、并行和条件执行，适用于复杂文档分析等任务。LangChain社群活跃，开发了许多工具和插件，提升框架功能，如与外部数据库集成。未来，LangChain目标包括增强模型支持和资源管理，以扩展应用领域。', 'LangChain 支持多种语言模型，如 GPT 系列和 Claude，通过统一接口简化了开发者在不同模型间的切换过程。例如，可以在开发时用 GPT-4 测试，并在生产环境中使用更经济的开源模型。此外，LangChain 可集成数据库、API 和其他服务，使 AI 功能兼容现有系统，如与 CRM 系统结合自动生成客户报告。', 'LangChain为企业提供了安全及监控功能、丰富的文档和社区资源，便于开发者利用其构建人工智能应用；此外，包含个人信息。'], 'summary_ids': ['151651f9-8223-4d00-ba57-947426b1036f', '6769cbcb-a754-438b-888f-c7b103f934ea', '18f704b0-7921-418f-b38a-7dbd3abdf517', '9282f845-cec5-412e-abd9-eb576ef70d59'], 'parent_child_map': {'151651f9-8223-4d00-ba57-947426b1036f': ['b244097f-e171-4811-8279-67b16082db40', '5bf426e0-2176-4424-8d35-c835cb553523', 'fd0fbf9c-815d-45b4-a925-021fdb4e5a5f', 'ad2ca09b-a19f-4aaa-bb8f-4f70e5c6d538', 'a2a48024-b664-4e4a-8136-b5a705e43884', 'e84bd680-fef1-42c4-8215-d4db8a077e8e'], '6769cbcb-a754-438b-888f-c7b103f934ea': ['fa51cc37-2665-4d62-a3a8-4ad52af695a0', 'a59fa74e-fe67-4e3c-97c9-d28d96e039c9', 'bd45d0f2-7369-443c-977b-6e62f29aa6e8', '0574a965-0f2d-42a3-8fa4-9afb39cba54b', '8af5c05d-f769-4d88-b272-fc43aeefceca'], '18f704b0-7921-418f-b38a-7dbd3abdf517': ['c900e231-267f-49bd-b544-a26efc5cdaed', 'b0ecc3b3-753b-4bca-aff4-43fd47296cbc', '557bd4f1-89bf-493c-8e3a-d4160181b370'], '9282f845-cec5-412e-abd9-eb576ef70d59': ['86b24532-b010-4d72-8e90-2c5fd9de126e', 'b5786d7b-608f-4384-9820-7c663db9b08b', '482ac2d9-c2e0-4a37-845a-0cf88d9b5e26', '34d80142-06f6-4099-b363-ed13367ce49e']}, 'cluster_centroids': None, 'cluster_probabilities': None, 'doc_id_map': {'b244097f-e171-4811-8279-67b16082db40': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '5bf426e0-2176-4424-8d35-c835cb553523': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'fa51cc37-2665-4d62-a3a8-4ad52af695a0': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', 'a59fa74e-fe67-4e3c-97c9-d28d96e039c9': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'c900e231-267f-49bd-b544-a26efc5cdaed': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fd0fbf9c-815d-45b4-a925-021fdb4e5a5f': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', 'bd45d0f2-7369-443c-977b-6e62f29aa6e8': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '86b24532-b010-4d72-8e90-2c5fd9de126e': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'b0ecc3b3-753b-4bca-aff4-43fd47296cbc': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', 'ad2ca09b-a19f-4aaa-bb8f-4f70e5c6d538': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', 'b5786d7b-608f-4384-9820-7c663db9b08b': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '0574a965-0f2d-42a3-8fa4-9afb39cba54b': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', '557bd4f1-89bf-493c-8e3a-d4160181b370': '我是崔鹏', 'a2a48024-b664-4e4a-8136-b5a705e43884': '崔鹏', '482ac2d9-c2e0-4a37-845a-0cf88d9b5e26': 'sxy', 'e84bd680-fef1-42c4-8215-d4db8a077e8e': '崔鹏是sxy的爸爸', '34d80142-06f6-4099-b363-ed13367ce49e': 'sxy是崔鹏的女儿', '8af5c05d-f769-4d88-b272-fc43aeefceca': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '151651f9-8223-4d00-ba57-947426b1036f': 'LangChain是一个开发基于语言模型的应用框架，具有提示管理、记忆管理和模型链接等核心功能，助力聊天机器人、文档分析工具和知识库等应用的实现，其灵活性适合多领域场景。', '6769cbcb-a754-438b-888f-c7b103f934ea': 'LCEL（LangChain Expression Language）是LangChain的重要特性，提供功能类似于管道操作的声明式方法，方便开发者组合组件和创建复杂的工作流。LCEL支持多种执行模式，如串行、并行和条件执行，适用于复杂文档分析等任务。LangChain社群活跃，开发了许多工具和插件，提升框架功能，如与外部数据库集成。未来，LangChain目标包括增强模型支持和资源管理，以扩展应用领域。', '18f704b0-7921-418f-b38a-7dbd3abdf517': 'LangChain 支持多种语言模型，如 GPT 系列和 Claude，通过统一接口简化了开发者在不同模型间的切换过程。例如，可以在开发时用 GPT-4 测试，并在生产环境中使用更经济的开源模型。此外，LangChain 可集成数据库、API 和其他服务，使 AI 功能兼容现有系统，如与 CRM 系统结合自动生成客户报告。', '9282f845-cec5-412e-abd9-eb576ef70d59': 'LangChain为企业提供了安全及监控功能、丰富的文档和社区资源，便于开发者利用其构建人工智能应用；此外，包含个人信息。', 'cf566af5-11a9-437d-992a-da4fb89fdca0': 'LangChain 是一个用于开发语言模型应用的框架，具备提示管理、记忆管理和模型链接功能，支持聊天机器人和文档分析等诸多应用。其 LCEL 功能类似管道操作，允许声明式组合组件及创建复杂工作流，支持多种执行模式。LangChain 社区活跃，开发了众多工具和插件，如与外部数据库集成，以扩展框架功能。它简化了多模型间的切换过程并支持多种语言模型，通过数据库和 API 的集成，AI 功能可适应现有系统。LangChain 提供企业级安全、监控功能及丰富资源，便于开发者创建 AI 应用。'}}, 
      
      
      
      1: {'docs': ['LangChain是一个开发基于语言模型的应用框架，具有提示管理、记忆管理和模型链接等核心功能，助力聊天机器人、文档分析工具和知识库等应用的实现，其灵活性适合多领域场景。', 'LCEL（LangChain Expression Language）是LangChain的重要特性，提供功能类似于管道操作的声明式方法，方便开发者组合组件和创建复杂的工作流。LCEL支持多种执行模式，如串行、并行和条件执行，适用于复杂文档分析等任务。LangChain社群活跃，开发了许多工具和插件，提升框架功能，如与外部数据库集成。未来，LangChain目标包括增强模型支持和资源管理，以扩展应用领域。', 'LangChain 支持多种语言模型，如 GPT 系列和 Claude，通过统一接口简化了开发者在不同模型间的切换过程。例如，可以在开发时用 GPT-4 测试，并在生产环境中使用更经济的开源模型。此外，LangChain 可集成数据库、API 和其他服务，使 AI 功能兼容现有系统，如与 CRM 系统结合自动生成客户报告。', 'LangChain为企业提供了安全及监控功能、丰富的文档和社区资源，便于开发者利用其构建人工智能应用；此外，包含个人信息。'], 'doc_ids': ['151651f9-8223-4d00-ba57-947426b1036f', '6769cbcb-a754-438b-888f-c7b103f934ea', '18f704b0-7921-418f-b38a-7dbd3abdf517', '9282f845-cec5-412e-abd9-eb576ef70d59'], 'summaries': ['LangChain 是一个用于开发语言模型应用的框架，具备提示管理、记忆管理和模型链接功能，支持聊天机器人和文档分析等诸多应用。其 LCEL 功能类似管道操作，允许声明式组合组件及创建复杂工作流，支持多种执行模式。LangChain 社区活跃，开发了众多工具和插件，如与外部数据库集成，以扩展框架功能。它简化了多模型间的切换过程并支持多种语言模型，通过数据库和 API 的集成，AI 功能可适应现有系统。LangChain 提供企业级安全、监控功能及丰富资源，便于开发者创建 AI 应用。'], 'summary_ids': ['cf566af5-11a9-437d-992a-da4fb89fdca0'], 'parent_child_map': {'cf566af5-11a9-437d-992a-da4fb89fdca0': ['151651f9-8223-4d00-ba57-947426b1036f', '6769cbcb-a754-438b-888f-c7b103f934ea', '18f704b0-7921-418f-b38a-7dbd3abdf517', '9282f845-cec5-412e-abd9-eb576ef70d59']}, 'cluster_centroids': None, 'cluster_probabilities': None, 'doc_id_map': {'b244097f-e171-4811-8279-67b16082db40': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '5bf426e0-2176-4424-8d35-c835cb553523': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'fa51cc37-2665-4d62-a3a8-4ad52af695a0': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', 'a59fa74e-fe67-4e3c-97c9-d28d96e039c9': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'c900e231-267f-49bd-b544-a26efc5cdaed': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fd0fbf9c-815d-45b4-a925-021fdb4e5a5f': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', 'bd45d0f2-7369-443c-977b-6e62f29aa6e8': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '86b24532-b010-4d72-8e90-2c5fd9de126e': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'b0ecc3b3-753b-4bca-aff4-43fd47296cbc': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', 'ad2ca09b-a19f-4aaa-bb8f-4f70e5c6d538': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', 'b5786d7b-608f-4384-9820-7c663db9b08b': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '0574a965-0f2d-42a3-8fa4-9afb39cba54b': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', '557bd4f1-89bf-493c-8e3a-d4160181b370': '我是崔鹏', 'a2a48024-b664-4e4a-8136-b5a705e43884': '崔鹏', '482ac2d9-c2e0-4a37-845a-0cf88d9b5e26': 'sxy', 'e84bd680-fef1-42c4-8215-d4db8a077e8e': '崔鹏是sxy的爸爸', '34d80142-06f6-4099-b363-ed13367ce49e': 'sxy是崔鹏的女儿', '8af5c05d-f769-4d88-b272-fc43aeefceca': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '151651f9-8223-4d00-ba57-947426b1036f': 'LangChain是一个开发基于语言模型的应用框架，具有提示管理、记忆管理和模型链接等核心功能，助力聊天机器人、文档分析工具和知识库等应用的实现，其灵活性适合多领域场景。', '6769cbcb-a754-438b-888f-c7b103f934ea': 'LCEL（LangChain Expression Language）是LangChain的重要特性，提供功能类似于管道操作的声明式方法，方便开发者组合组件和创建复杂的工作流。LCEL支持多种执行模式，如串行、并行和条件执行，适用于复杂文档分析等任务。LangChain社群活跃，开发了许多工具和插件，提升框架功能，如与外部数据库集成。未来，LangChain目标包括增强模型支持和资源管理，以扩展应用领域。', '18f704b0-7921-418f-b38a-7dbd3abdf517': 'LangChain 支持多种语言模型，如 GPT 系列和 Claude，通过统一接口简化了开发者在不同模型间的切换过程。例如，可以在开发时用 GPT-4 测试，并在生产环境中使用更经济的开源模型。此外，LangChain 可集成数据库、API 和其他服务，使 AI 功能兼容现有系统，如与 CRM 系统结合自动生成客户报告。', '9282f845-cec5-412e-abd9-eb576ef70d59': 'LangChain为企业提供了安全及监控功能、丰富的文档和社区资源，便于开发者利用其构建人工智能应用；此外，包含个人信息。', 'cf566af5-11a9-437d-992a-da4fb89fdca0': 'LangChain 是一个用于开发语言模型应用的框架，具备提示管理、记忆管理和模型链接功能，支持聊天机器人和文档分析等诸多应用。其 LCEL 功能类似管道操作，允许声明式组合组件及创建复杂工作流，支持多种执行模式。LangChain 社区活跃，开发了众多工具和插件，如与外部数据库集成，以扩展框架功能。它简化了多模型间的切换过程并支持多种语言模型，通过数据库和 API 的集成，AI 功能可适应现有系统。LangChain 提供企业级安全、监控功能及丰富资源，便于开发者创建 AI 应用。'}}}

a.keys(), type(a[0]), type(a[1]), a[0].keys(), a[1].keys(), len(a[0]["docs"]), len(a[1]["docs"]), a[0]["parent_child_map"], "\n\n", a[1]["parent_child_map"], type(a.values()), len(a.values()), len(a[0]["doc_ids"]), len(a[1]["doc_ids"])

(dict_keys([0, 1]),
 dict,
 dict,
 dict_keys(['docs', 'doc_ids', 'summaries', 'summary_ids', 'parent_child_map', 'cluster_centroids', 'cluster_probabilities', 'doc_id_map']),
 dict_keys(['docs', 'doc_ids', 'summaries', 'summary_ids', 'parent_child_map', 'cluster_centroids', 'cluster_probabilities', 'doc_id_map']),
 18,
 4,
 {'151651f9-8223-4d00-ba57-947426b1036f': ['b244097f-e171-4811-8279-67b16082db40',
   '5bf426e0-2176-4424-8d35-c835cb553523',
   'fd0fbf9c-815d-45b4-a925-021fdb4e5a5f',
   'ad2ca09b-a19f-4aaa-bb8f-4f70e5c6d538',
   'a2a48024-b664-4e4a-8136-b5a705e43884',
   'e84bd680-fef1-42c4-8215-d4db8a077e8e'],
  '6769cbcb-a754-438b-888f-c7b103f934ea': ['fa51cc37-2665-4d62-a3a8-4ad52af695a0',
   'a59fa74e-fe67-4e3c-97c9-d28d96e039c9',
   'bd45d0f2-7369-443c-977b-6e62f29aa6e8',
   '0574a965-0f2d-42a3-8fa4-9afb39cba54b',
   '8af5c05d-f769-4d88-b272-fc43aeefceca'],
  '18f704b0-7921-418f-b38a-7dbd3abdf517': ['c900e231-267f-49bd-b544-a26efc5cdaed',
   'b0ecc3b3-753b-4bca-aff4-43fd47296

In [1]:
chunks = [
    "LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。",
    "LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。",
    "LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。",
    "使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。",
    "LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。",
    "在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。",
    "LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。",
    "对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。",
    "LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。",
    "LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。",
    "LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。",
    "LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。",
    "我是崔鹏",
    "崔鹏",
    "sxy",
    "崔鹏是sxy的爸爸",
    "sxy是崔鹏的女儿",
    "崔鹏是sxy的爸爸，sxy是崔鹏的女儿"
]

In [ ]:
len(chunks)

In [5]:
hierarchy = {0: {'docs': ['LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', '我是崔鹏', '崔鹏', 'sxy', '崔鹏是sxy的爸爸', 'sxy是崔鹏的女儿', '崔鹏是sxy的爸爸，sxy是崔鹏的女儿'], 'doc_ids': ['7aecad58-3bb4-4f1c-aff8-efb2b5a408b8', '03362429-c06a-4dd6-8f5f-83edb8fadb06', '264602ce-c0ff-420e-a050-ec1cb11d7cb3', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc', 'fc7adc44-82cf-4775-97c5-6b7209ec1375', '5ed80d05-a17c-47d6-876f-a8610a739b90', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658', 'ba2d1e70-9416-4f8e-b682-d0389e855e88', '348ed288-99f8-4da8-b4b5-53609f16f287', '25142322-0dfc-4e64-b458-0750d4c7cc6f', '34adaada-f3c9-45f4-b150-bfd1d25ac474', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb', '203ee849-5fda-4f9b-92d5-55fcde65804f', '146f06f5-3f32-4322-bad0-cc0d44d925e7', '7f282e49-dfda-4b56-adf8-417e5155cc43', '38c0387e-b84e-4d5c-b571-1b6273fe198e'], 'summaries': ['LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。'], 'summary_ids': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938'], 'parent_child_map': {'812952d0-bec4-4299-982b-d8fc8514ccc0': ['7aecad58-3bb4-4f1c-aff8-efb2b5a408b8', '264602ce-c0ff-420e-a050-ec1cb11d7cb3', 'ba2d1e70-9416-4f8e-b682-d0389e855e88', '348ed288-99f8-4da8-b4b5-53609f16f287', '146f06f5-3f32-4322-bad0-cc0d44d925e7'], '5454ab2c-8a4c-45ae-be33-1806ab1921f5': ['03362429-c06a-4dd6-8f5f-83edb8fadb06', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328', 'fc7adc44-82cf-4775-97c5-6b7209ec1375', '5ed80d05-a17c-47d6-876f-a8610a739b90', '34adaada-f3c9-45f4-b150-bfd1d25ac474', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb'], '4fd70814-fc53-4879-825c-ec6aefc64938': ['dd5c9aca-06e9-49dd-834b-3de42ff0d1fc', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658', '25142322-0dfc-4e64-b458-0750d4c7cc6f', '203ee849-5fda-4f9b-92d5-55fcde65804f', '7f282e49-dfda-4b56-adf8-417e5155cc43', '38c0387e-b84e-4d5c-b571-1b6273fe198e']}, 'cluster_centroids': None, 'cluster_probabilities': None, 'doc_id_map': {'7aecad58-3bb4-4f1c-aff8-efb2b5a408b8': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '03362429-c06a-4dd6-8f5f-83edb8fadb06': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', '264602ce-c0ff-420e-a050-ec1cb11d7cb3': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fc7adc44-82cf-4775-97c5-6b7209ec1375': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', '5ed80d05-a17c-47d6-876f-a8610a739b90': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'ba2d1e70-9416-4f8e-b682-d0389e855e88': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', '348ed288-99f8-4da8-b4b5-53609f16f287': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', '25142322-0dfc-4e64-b458-0750d4c7cc6f': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '34adaada-f3c9-45f4-b150-bfd1d25ac474': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9': '我是崔鹏', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb': '崔鹏', '203ee849-5fda-4f9b-92d5-55fcde65804f': 'sxy', '146f06f5-3f32-4322-bad0-cc0d44d925e7': '崔鹏是sxy的爸爸', '7f282e49-dfda-4b56-adf8-417e5155cc43': 'sxy是崔鹏的女儿', '38c0387e-b84e-4d5c-b571-1b6273fe198e': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '812952d0-bec4-4299-982b-d8fc8514ccc0': 'LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', '5454ab2c-8a4c-45ae-be33-1806ab1921f5': 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', '4fd70814-fc53-4879-825c-ec6aefc64938': 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。', '7ede8dab-46f2-40b2-b6eb-b65560d7af55': 'LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'}}, 
             
             
             
1: {'docs': ['LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。'], 'doc_ids': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938'], 'summaries': ['LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'], 'summary_ids': ['7ede8dab-46f2-40b2-b6eb-b65560d7af55'], 'parent_child_map': {'7ede8dab-46f2-40b2-b6eb-b65560d7af55': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938']}, 'cluster_centroids': None, 'cluster_probabilities': None, 'doc_id_map': {'7aecad58-3bb4-4f1c-aff8-efb2b5a408b8': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '03362429-c06a-4dd6-8f5f-83edb8fadb06': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', '264602ce-c0ff-420e-a050-ec1cb11d7cb3': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fc7adc44-82cf-4775-97c5-6b7209ec1375': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', '5ed80d05-a17c-47d6-876f-a8610a739b90': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'ba2d1e70-9416-4f8e-b682-d0389e855e88': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', '348ed288-99f8-4da8-b4b5-53609f16f287': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', '25142322-0dfc-4e64-b458-0750d4c7cc6f': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '34adaada-f3c9-45f4-b150-bfd1d25ac474': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9': '我是崔鹏', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb': '崔鹏', '203ee849-5fda-4f9b-92d5-55fcde65804f': 'sxy', '146f06f5-3f32-4322-bad0-cc0d44d925e7': '崔鹏是sxy的爸爸', '7f282e49-dfda-4b56-adf8-417e5155cc43': 'sxy是崔鹏的女儿', '38c0387e-b84e-4d5c-b571-1b6273fe198e': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '812952d0-bec4-4299-982b-d8fc8514ccc0': 'LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', '5454ab2c-8a4c-45ae-be33-1806ab1921f5': 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', '4fd70814-fc53-4879-825c-ec6aefc64938': 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。', '7ede8dab-46f2-40b2-b6eb-b65560d7af55': 'LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'}}}


In [ ]:
type(hierarchy.values()), len(hierarchy.values())

In [ ]:
hierarchy.keys(), type(hierarchy[0]), hierarchy[0].keys(), type(hierarchy[1]), hierarchy[1].keys()

{0: 
{
    'docs': ['LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', '我是崔鹏', '崔鹏', 'sxy', '崔鹏是sxy的爸爸', 'sxy是崔鹏的女儿', '崔鹏是sxy的爸爸，sxy是崔鹏的女儿'], 

    'doc_ids': ['7aecad58-3bb4-4f1c-aff8-efb2b5a408b8', '03362429-c06a-4dd6-8f5f-83edb8fadb06', '264602ce-c0ff-420e-a050-ec1cb11d7cb3', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc', 'fc7adc44-82cf-4775-97c5-6b7209ec1375', '5ed80d05-a17c-47d6-876f-a8610a739b90', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658', 'ba2d1e70-9416-4f8e-b682-d0389e855e88', '348ed288-99f8-4da8-b4b5-53609f16f287', '25142322-0dfc-4e64-b458-0750d4c7cc6f', '34adaada-f3c9-45f4-b150-bfd1d25ac474', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb', '203ee849-5fda-4f9b-92d5-55fcde65804f', '146f06f5-3f32-4322-bad0-cc0d44d925e7', '7f282e49-dfda-4b56-adf8-417e5155cc43', '38c0387e-b84e-4d5c-b571-1b6273fe198e'], 
    
    'summaries': ['LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。'], 
    
    'summary_ids': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938'], 
    
    'parent_child_map': {'812952d0-bec4-4299-982b-d8fc8514ccc0': ['7aecad58-3bb4-4f1c-aff8-efb2b5a408b8', '264602ce-c0ff-420e-a050-ec1cb11d7cb3', 'ba2d1e70-9416-4f8e-b682-d0389e855e88', '348ed288-99f8-4da8-b4b5-53609f16f287', '146f06f5-3f32-4322-bad0-cc0d44d925e7'], '5454ab2c-8a4c-45ae-be33-1806ab1921f5': ['03362429-c06a-4dd6-8f5f-83edb8fadb06', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328', 'fc7adc44-82cf-4775-97c5-6b7209ec1375', '5ed80d05-a17c-47d6-876f-a8610a739b90', '34adaada-f3c9-45f4-b150-bfd1d25ac474', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb'], '4fd70814-fc53-4879-825c-ec6aefc64938': ['dd5c9aca-06e9-49dd-834b-3de42ff0d1fc', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658', '25142322-0dfc-4e64-b458-0750d4c7cc6f', '203ee849-5fda-4f9b-92d5-55fcde65804f', '7f282e49-dfda-4b56-adf8-417e5155cc43', '38c0387e-b84e-4d5c-b571-1b6273fe198e']}, 
    
    'cluster_centroids': None, 
    
    'cluster_probabilities': None, 
    
    'doc_id_map': {'7aecad58-3bb4-4f1c-aff8-efb2b5a408b8': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '03362429-c06a-4dd6-8f5f-83edb8fadb06': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', '264602ce-c0ff-420e-a050-ec1cb11d7cb3': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fc7adc44-82cf-4775-97c5-6b7209ec1375': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', '5ed80d05-a17c-47d6-876f-a8610a739b90': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'ba2d1e70-9416-4f8e-b682-d0389e855e88': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', '348ed288-99f8-4da8-b4b5-53609f16f287': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', '25142322-0dfc-4e64-b458-0750d4c7cc6f': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '34adaada-f3c9-45f4-b150-bfd1d25ac474': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9': '我是崔鹏', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb': '崔鹏', '203ee849-5fda-4f9b-92d5-55fcde65804f': 'sxy', '146f06f5-3f32-4322-bad0-cc0d44d925e7': '崔鹏是sxy的爸爸', '7f282e49-dfda-4b56-adf8-417e5155cc43': 'sxy是崔鹏的女儿', '38c0387e-b84e-4d5c-b571-1b6273fe198e': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '812952d0-bec4-4299-982b-d8fc8514ccc0': 'LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', '5454ab2c-8a4c-45ae-be33-1806ab1921f5': 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', '4fd70814-fc53-4879-825c-ec6aefc64938': 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。', '7ede8dab-46f2-40b2-b6eb-b65560d7af55': 'LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'}}, 
    
1: 
{
    'docs': ['LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。'], 
    
    'doc_ids': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938'], 
    
    'summaries': ['LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'], 
    
    'summary_ids': ['7ede8dab-46f2-40b2-b6eb-b65560d7af55'], 
    
    'parent_child_map': {'7ede8dab-46f2-40b2-b6eb-b65560d7af55': ['812952d0-bec4-4299-982b-d8fc8514ccc0', '5454ab2c-8a4c-45ae-be33-1806ab1921f5', '4fd70814-fc53-4879-825c-ec6aefc64938']}, 
    
    'cluster_centroids': None, 'cluster_probabilities': None, 
    
    'doc_id_map': {'7aecad58-3bb4-4f1c-aff8-efb2b5a408b8': 'LangChain是一个强大的框架，用于开发基于语言模型的应用。它提供了丰富的组件和工具，可以轻松构建复杂的AI应用。例如，开发者可以利用LangChain快速搭建一个智能客服系统，或者创建一个自动化文档处理工具。', '03362429-c06a-4dd6-8f5f-83edb8fadb06': 'LangChain的核心功能包括提示管理、记忆管理和模型链接。提示管理帮助开发者更好地控制与语言模型的交互，例如通过动态生成提示词来优化模型输出。记忆管理则允许应用记住用户的上下文信息，从而实现更连贯的对话体验。', '264602ce-c0ff-420e-a050-ec1cb11d7cb3': 'LCEL（LangChain Expression Language）是LangChain的一个重要特性。它提供了一种声明式的方式来组合不同的组件，类似于函数式编程中的管道操作。通过LCEL，开发者可以轻松定义复杂的工作流，例如将多个模型串联起来处理数据。', '9a769e7a-c9a0-4f4b-a1c8-19055cad5328': '使用LCEL，开发者可以轻松地创建复杂的处理流程。它支持串行、并行和条件执行等多种模式。例如，在一个文档分析任务中，可以先用一个模型提取关键信息，再用另一个模型进行分类，最后根据分类结果执行不同的操作。', 'dd5c9aca-06e9-49dd-834b-3de42ff0d1fc': 'LangChain支持多种语言模型，包括GPT系列、Claude和开源模型。它还提供了统一的接口来管理这些模型，使得开发者可以无缝切换不同的模型。例如，开发者可以在开发阶段使用GPT-4进行测试，而在生产环境中切换到成本更低的开源模型。', 'fc7adc44-82cf-4775-97c5-6b7209ec1375': '在实际应用中，LangChain可以用于构建聊天机器人、文档分析工具和知识库系统等。它的模块化设计使得代码复用变得简单。例如，一个聊天机器人可以复用已有的记忆管理模块，从而快速实现上下文感知功能。', '5ed80d05-a17c-47d6-876f-a8610a739b90': 'LangChain的生态系统正在快速发展，社区贡献了大量的工具和插件。这些扩展进一步增强了框架的功能。例如，社区开发的插件可以支持与外部数据库的集成，或者提供更高级的提示词优化功能。', '8e6824cd-0801-4dc1-8c87-5f4a7f10c658': '对于企业应用，LangChain提供了完善的安全和监控功能。它可以跟踪模型调用、管理API密钥并记录详细的日志。例如，企业可以通过LangChain的监控功能分析模型的使用情况，优化资源分配并确保数据安全。', 'ba2d1e70-9416-4f8e-b682-d0389e855e88': 'LangChain还支持与外部工具的集成，例如数据库、API和第三方服务。这使得开发者可以轻松地将AI功能嵌入到现有的系统中。例如，可以将LangChain与CRM系统集成，自动生成客户报告。', '348ed288-99f8-4da8-b4b5-53609f16f287': 'LangChain的灵活性使其适用于多种场景，包括教育、医疗、金融等领域。例如，在教育领域，LangChain可以用于构建智能辅导系统，帮助学生更好地理解复杂的概念。', '25142322-0dfc-4e64-b458-0750d4c7cc6f': 'LangChain的文档和社区资源非常丰富，开发者可以快速上手并解决遇到的问题。官方提供了详细的教程和示例代码，帮助开发者从零开始构建AI应用。', '34adaada-f3c9-45f4-b150-bfd1d25ac474': 'LangChain的未来发展方向包括更强大的模型支持、更高效的资源管理和更广泛的应用场景。例如，未来的版本可能会支持更多的开源模型，并提供更细粒度的资源控制功能。', 'd2c4b849-0a5a-41ce-945d-18d6484bf2b9': '我是崔鹏', 'a65b47dc-4cd7-4cc8-bf8e-928b978db8bb': '崔鹏', '203ee849-5fda-4f9b-92d5-55fcde65804f': 'sxy', '146f06f5-3f32-4322-bad0-cc0d44d925e7': '崔鹏是sxy的爸爸', '7f282e49-dfda-4b56-adf8-417e5155cc43': 'sxy是崔鹏的女儿', '38c0387e-b84e-4d5c-b571-1b6273fe198e': '崔鹏是sxy的爸爸，sxy是崔鹏的女儿', '812952d0-bec4-4299-982b-d8fc8514ccc0': 'LangChain是一个用于开发基于语言模型的应用的框架，提供多种组件和工具，支持创建智能客服和自动化文档处理系统。LCEL（LangChain Expression Language）允许开发者通过声明式方式轻松组合组件，涉及复杂工作流的定义。该框架还支持与外部工具如数据库、API等集成，使得AI功能可以嵌入至现有系统。LangChain适用于教育、医疗和金融等领域，例如，可以构建智能辅导系统帮助学生理解复杂概念。', '5454ab2c-8a4c-45ae-be33-1806ab1921f5': 'LangChain核心功能包括提示管理、记忆管理和模型链接，旨在增强与语言模型的交互体验。提示管理通过动态生成提示词优化模型输出，而记忆管理则保存用户的上下文信息以实现连贯对话。它支持创建复杂处理流程，适用于聊天机器人、文档分析工具等应用。其模块化设计提高了代码复用率，社区贡献增强了框架功能，未来将着重于模型支持、资源管理及应用扩展。', '4fd70814-fc53-4879-825c-ec6aefc64938': 'LangChain支持多种语言模型，如GPT系列和Claude，并提供统一接口以方便模型切换。开发者可以在不同阶段选择合适的模型，以优化成本。它还具备安全和监控功能，能够跟踪模型调用和管理API密钥，帮助企业分析模型使用情况，确保数据安全。此外，LangChain提供详细文档和丰富社区资源，助力开发者快速上手和解决问题。', '7ede8dab-46f2-40b2-b6eb-b65560d7af55': 'LangChain是一个开发基于语言模型应用的框架，支持组件组合和与外部工具集成，适用于教育、医疗和金融等领域。其核心功能包括提示管理、记忆管理和模型链接，优化与语言模型的交互体验，并能创建智能客服和文档处理系统。LangChain具备模块化设计和多种语言模型支持，允许开发者选择合适的模型以优化成本，且提供安全监控功能，确保数据安全。此外，丰富的文档和社区资源助力开发者快速上手。'}}}
